In [1]:
import pandas as pd
import json
#Print 
# 假设你的JSON数据存储在一个变量中
json_data = []
    # ... 其他数据
filename='/Users/yitengxu/Desktop/PaperScraper-Final/result_collected/combined_JSON/raw_combined_all.json'

# 你还可以执行许多其他操作，例如排序、分组、可视化等


In [2]:
import re
import json

def is_valid(entry):
    # 检查每个字段，如果至少有一个字段不是"N/A"或其他无效值，则返回True
    for key, value in entry.items():
        if key != "Absorber Layer" and value not in ["N/A", "????", "<Value in %>%", "<Value in V>V", "<Value in mA/cm2>mA/cm2", "<Value in %>%"]:
            if any(char.isdigit() for char in value):
                return True
    return False

def preprocess_data(Raw_Json_data):
    valid_data = []
    invalid_data = []

    for entry in Raw_Json_data:
        if is_valid(entry):
            valid_data.append(entry)
        else:
            invalid_data.append(entry)

    return valid_data, invalid_data



def process_data(json_data):
    processed_data = []
    for item in json_data:
        valid_item = True
        # 检查 "Absorber Layer" 是否为字符串
        if not isinstance(item['Absorber Layer'], str):
            item['Absorber Layer'] = None
            valid_item = False

        # 检查其他字段是否为数字加单位，可能包括误差
        for key in ['PCE', 'Voc', 'Jsc', 'FF']:
            if key in item:
                value = item[key]
                # 用于匹配多个值的正则表达式
                matches = re.findall(r'(\d+\.?\d*)\s*(?:±\s*(\d+\.?\d*)\s*)?(%|V|mV|mA/cm2)?', value)
                if matches:
                    # 处理每个匹配的值
                    for match in matches:
                        value, error, unit = match
                        value = float(value)
                        error = float(error) if error else None

                        # 转换Voc的单位
                        if key == 'Voc' and unit == 'mV':
                            value /= 1000
                            if error:
                                error /= 1000
                            unit = 'V'

                        if key == 'Voc' and unit == '':
                            unit = 'V'

                        if key == 'PCE' and unit == '':                        
                            unit = '%'
                            if value < 1:
                                value *=100
                                
                        if key == 'Jsc' and unit == '':                       
                            unit = 'mA/cm2'

                        # 可以在此处添加其他单位转换逻辑
                            # 转换Jsc的单位
                        if key == 'Jsc' and 'μA' in unit:
                            value /= 1000
                            if error:
                                error /= 1000
                            unit = 'mA/cm2'

                        # 处理FF的单位
                        if key == 'FF' :
                    
                            unit = '%'
                            if value < 1:
                                value *= 100


                        item[key] = {'value': value, 'error': error, 'unit': unit}
                else:
                    item[key] = None
                    valid_item = False

        # 如果所有字段都为null，则不保存此数据
        if valid_item or any(item[key] for key in ['PCE', 'Voc', 'Jsc', 'FF']):
            processed_data.append(item)

    return processed_data

# 读取JSON文件
filename='/Users/yitengxu/Desktop/PaperScraper-Final/result_collected/combined_JSON/raw_combined_all.json'
Raw_Json_data = []

with open(filename, 'r') as file:
    Raw_Json_data = json.load(file)

# 预处理数据，分类有效和无效信息
valid_data, invalid_data = preprocess_data(Raw_Json_data)

# 处理有效数据，提取数字和单位
processed_data = process_data(valid_data)



In [3]:
toprint=pd.DataFrame(processed_data)
print(toprint)



                    Absorber Layer  \
0                   PTB7-Th:PC70BM   
1     PTB7-Th:P(BDTSi–DFBT):PC70BM   
2                              N/A   
3                           CZTSSe   
4                       Perovskite   
...                            ...   
3117                        COi6IC   
3118                       COi6FIC   
3119                      COi6DFIC   
3120                           IB3   
3121                           IB4   

                                               PCE  \
0      {'value': 8.49, 'error': None, 'unit': '%'}   
1     {'value': 10.05, 'error': None, 'unit': '%'}   
2      {'value': 6.86, 'error': None, 'unit': '%'}   
3      {'value': 5.66, 'error': None, 'unit': '%'}   
4      {'value': 22.5, 'error': None, 'unit': '%'}   
...                                            ...   
3117   {'value': 8.43, 'error': None, 'unit': '%'}   
3118   {'value': 9.12, 'error': None, 'unit': '%'}   
3119   {'value': 8.25, 'error': None, 'unit': '%'}   
312

In [4]:
Raw=pd.DataFrame(Raw_Json_data)
print(Raw)

                    Absorber Layer  \
0                   PTB7-Th:PC70BM   
1     PTB7-Th:P(BDTSi–DFBT):PC70BM   
2                              N/A   
3                           CZTSSe   
4                       Perovskite   
...                            ...   
3620                        COi6IC   
3621                       COi6FIC   
3622                      COi6DFIC   
3623                           IB3   
3624                           IB4   

                                               PCE  \
0      {'value': 8.49, 'error': None, 'unit': '%'}   
1     {'value': 10.05, 'error': None, 'unit': '%'}   
2      {'value': 6.86, 'error': None, 'unit': '%'}   
3      {'value': 5.66, 'error': None, 'unit': '%'}   
4      {'value': 22.5, 'error': None, 'unit': '%'}   
...                                            ...   
3620   {'value': 8.43, 'error': None, 'unit': '%'}   
3621   {'value': 9.12, 'error': None, 'unit': '%'}   
3622   {'value': 8.25, 'error': None, 'unit': '%'}   
362

In [5]:
#save model
import json
toprint.to_csv('/Users/yitengxu/Desktop/PaperScraper-Final/result_collected/cleared_all.csv', index=False)


def save_to_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# 示例用法
file_path = '/Users/yitengxu/Desktop/PaperScraper-Final/result_collected/combined_JSON/combined_cleared_all.json'
save_to_json(processed_data, file_path)
